In [1]:
# coding=utf-8
'''
日产外观颜色匹配
通过已经匹配的车型ID查询该车型ID下所有可能出现的外观颜色信息
'''

import pymysql

def select_hash(model_id):
    '''车型ID 查 hash'''
    conn = pymysql.connect(host='192.168.3.114', user='jing', passwd='123456', db='vin_nissan', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE vin_nissan")
    sql = '''
        SELECT GROUP_CONCAT(vin_group_hash) 
        FROM `sub_config_hash_map` 
        WHERE match_ids LIKE "%{}%";
    '''.format(model_id)
    cur.execute(sql)
    result = cur.fetchall()
    result_list = []
    if result:
        result_list = result[0][0].split(',')
    cur.close()
    conn.close()
    return result_list

def select_vin(hash_list):
    '''通过 hash 查 VIN'''
    conn = pymysql.connect(host='192.168.3.114', user='jing', passwd='123456', db='vin_nissan', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE vin_nissan")
    sql = '''
        SELECT `sub_vin_parts_hash`.vin
        FROM `sub_vin_parts_hash` 
        WHERE `sub_vin_parts_hash`.parts_hash IN {};
    '''.format(tuple(hash_list))
    cur.execute(sql)
    result = cur.fetchall()
    result_list = []
    if result:
        for i in result:
            result_list.append(i[0])
    cur.close()
    conn.close()
    return result_list

def select_vin_color_info(vin_list):
    '''通过 hash 查 VIN 外观颜色信息'''
    conn = pymysql.connect(host='192.168.3.114', user='jing', passwd='123456', db='vin_nissan', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE vin_nissan")
    sql = '''
        SELECT T2.*,`nissan_body_color`.name
        FROM
            (SELECT 
                `app_nissan_vin`.vin, `app_nissan_vin`.date,
                REPLACE(`app_nissan_vin`.color,' PRC','') AS 'color_code',
                MIN(`app_nissan_vin`.date) AS 'MIN_date', MAX(`app_nissan_vin`.date) AS 'MAX_date',
                COUNT(*)
            FROM `app_nissan_vin`
            WHERE `app_nissan_vin`.vin IN {}
            GROUP BY color_code
            ORDER BY COUNT(*) DESC) AS T2
        LEFT JOIN `nissan_body_color`
        ON T2.color_code = `nissan_body_color`.code;
    '''.format(tuple(vin_list))
    cur.execute(sql)
    result = cur.fetchall()
    result_list = []
    if result:
        for i in result:
            print(i)
            result_list.append(i)
    cur.close()
    conn.close()
    return result_list

model_id = 3146
# 第一步，通过车型ID查可能的所有hash，并转化为列表，方便后续查询
hash_list = select_hash(model_id)
# 第二步，通过已查出的hash列表，查找所有的符合条件的VIN，
vin_list = select_vin(hash_list)
# 第三步，通过vin列表，给出所有VIN去重后的外观颜色信息
model_id_color = select_vin_color_info(vin_list)
print(model_id_color)



('LGBF1CE007R209903', '0706', 'B20', '0706', '0804', 612, '碧玉黑')
('LGBF1CE017R221574', '0710', 'B21', '0710', '0710', 2, '碧天蓝')
('LGBF1CE007R211585', '0707', 'B46', '0608', '0804', 222, '谧月蓝')
('LGBF1CE007R210033', '0706', 'D42', '0706', '0803', 41, '云雾绿')
('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '月光银')
('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '流光银')
('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '灰色')
('LGBF1CE007R215944', '0708', 'QX1', '0707', '0804', 39, '象牙白')
[('LGBF1CE007R209903', '0706', 'B20', '0706', '0804', 612, '碧玉黑'), ('LGBF1CE017R221574', '0710', 'B21', '0710', '0710', 2, '碧天蓝'), ('LGBF1CE007R211585', '0707', 'B46', '0608', '0804', 222, '谧月蓝'), ('LGBF1CE007R210033', '0706', 'D42', '0706', '0803', 41, '云雾绿'), ('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '月光银'), ('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '流光银'), ('LGBF1CE007R218293', '0709', 'K23', '0706', '0804', 189, '灰色'), ('LGBF1CE007R215944', '070

In [21]:
a = [['a',1],['c',3],['b',2]]
a = (('a',1),('c',3),('b',2))
b = sorted(a,key=lambda _:_[1],reverse=True)
print(a)
print(b)



(('a', 1), ('c', 3), ('b', 2))
[('c', 3), ('b', 2), ('a', 1)]
